In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
with np.load("notMNIST.npz") as data :
    Data, Target = data ["images"], data["labels"]
    posClass = 2
    negClass = 9
    dataIndx = (Target==posClass) + (Target==negClass)
    Data = Data[dataIndx]/255.
    Target = Target[dataIndx].reshape(-1, 1)
    Target[Target==posClass] = 1
    Target[Target==negClass] = 0
    np.random.seed(521)
    randIndx = np.arange(len(Data))
    np.random.shuffle(randIndx)
    Data, Target = Data[randIndx], Target[randIndx]
    trainData, trainTarget = Data[:3500], Target[:3500]
    validData, validTarget = Data[3500:3600], Target[3500:3600]
    testData, testTarget = Data[3600:], Target[3600:]

In [ ]:
trainData = np.reshape(trainData, [trainData.shape[0], 28*28])
validData = np.reshape(validData, [validData.shape[0], 28*28])
testData = np.reshape(testData, [testData.shape[0], 28*28])

In [ ]:
def mse_loss(yhat, target):
    loss = tf.reduce_mean(tf.pow(tf.subtract(yhat, target), 2))
    loss = tf.div(loss, 2)
    return loss

In [ ]:
#hyperparameters
# learning_rates = [0.005, 0.001, 0.0001]
learning_rates = [0.005]
mini_batch = 500
reg_coeff = 0.0
train_iter = 20000
train_loss = []

In [ ]:
for learning_rate in learning_rates:
    
    tf.reset_default_graph()
    train_loss_curr = []
    
    x = tf.placeholder(tf.float32, shape = [None, 784])
    y = tf.placeholder(tf.float32, shape = [None, 1])

    w = tf.Variable(np.random.randn())
    b = tf.cast(tf.Variable(0), dtype = tf.float32)
    yhat = tf.add(tf.multiply(x, w), b)

    mse = mse_loss(yhat, y)
    regul = tf.multiply(reg_coeff, tf.reduce_mean(tf.square(w)))
    cost = tf.add(mse, regul)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
    
    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)
    
    for epoch in range(train_iter):
        x_batch, y_batch = tf.train.batch([trainData, trainTarget], mini_batch)
        
        

        
        sess.run(optimizer, feed_dict = {x: x_batch, y:y_batch})
        w_curr, b_curr, c = sess.run([w, b, cost], feed_dict = {x: x_batch, y:y_batch})
        train_loss_curr.append(c)
        
        if epoch%3500 is 0:
            print("curr loss:", c)
            print("epoch:", epoch/3500)
            
    train_loss.append(train_loss_curr)
    
print("done")

In [ ]:
plotx = np.linspace(0, train_iter, train_iter)
ploty = np.array(train_loss)

for i in range(ploty.shape[0]):
    plt.plot(plotx, ploty[i])

plt.show()